In [1]:
# Install dependencies
import sys
!{sys.executable} -m pip install hdfs

In [2]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [3]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, DoubleType
from delta import *

In [4]:
# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [5]:
spark.sql(
    """
    SHOW DATABASES
    """
).show()

+-------------+
|    namespace|
+-------------+
|      default|
| pl7g1_silver|
|pl7g1_silver1|
|   reciclagem|
+-------------+



In [6]:
spark.sql(
    """
    Drop DATABASE PL7G1_Silver CASCADE
    """
)

DataFrame[]

In [7]:
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_SILVER.Reciclagem
    """
)

DataFrame[]

In [8]:
spark.sql(
    """
    CREATE DATABASE PL7G1_SILVER LOCATION 'hdfs://hdfs-nn:9000//Poluicao_ar/Silver/PL7G1_SILVER.db/'
    """
)

DataFrame[]

In [9]:
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_SILVER.Reciclagem
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_SILVER.Reciclagem (
        zone VARCHAR(50),
        district VARCHAR(50),
        fiscal_Month_Number INT,
        month_Name VARCHAR(31),
        diversion_Rate_Total DOUBLE,
        capture_Rate_Paper DOUBLE,
        capture_Rate_Mgp DOUBLE,
        capture_Rate_Total DOUBLE
        
    )
    USING DELTA
    PARTITIONED BY (
        fiscal_Year INT   
    )
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db/Reciclagem'
    """
)

DataFrame[]

In [10]:
spark.sql(
    """
    SHOW TABLES FROM PL7G1_SILVER
    """
).show()

+------------+----------+-----------+
|   namespace| tableName|isTemporary|
+------------+----------+-----------+
|pl7g1_silver|reciclagem|      false|
+------------+----------+-----------+



In [11]:
spark.sql(
    """
    SELECT *
    FROM PL7G1_SILVER.Reciclagem
    """
).show()

+----+--------+-------------------+----------+--------------------+------------------+----------------+------------------+-----------+
|zone|district|fiscal_Month_Number|month_Name|diversion_Rate_Total|capture_Rate_Paper|capture_Rate_Mgp|capture_Rate_Total|fiscal_Year|
+----+--------+-------------------+----------+--------------------+------------------+----------------+------------------+-----------+
+----+--------+-------------------+----------+--------------------+------------------+----------------+------------------+-----------+



In [12]:
spark.sql(
    """
    DESCRIBE FORMATTED PL7G1_SILVER.Reciclagem
    """
).toPandas()

col_name  \
0                           zone   
1                       district   
2            fiscal_Month_Number   
3                     month_Name   
4           diversion_Rate_Total   
5             capture_Rate_Paper   
6               capture_Rate_Mgp   
7             capture_Rate_Total   
8                    fiscal_Year   
9                                  
10                # Partitioning   
11                        Part 0   
12                                 
13  # Detailed Table Information   
14                          Name   
15                      Location   
16                      Provider   
17                         Owner   
18                      External   
19              Table Properties   

                                            data_type comment  
0                                              string          
1                                              string          
2                                                 int          
3                                              string          
4                                              double          
5                                              double          
6                                              double          
7                                              double          
8                                                 int          
9                                                              
10                                                             
11                                        fiscal_Year          
12                                                             
13                                                             
14                            pl7g1_silver.reciclagem          
15  hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_S...          
16                                              delta          
17                                             jovyan          
18                                               true          
19  [delta.minReaderVersion=1,delta.minWriterVersi...

In [13]:
#alterar
hdfs_path = "hdfs://hdfs-nn:9000//Poluicao_ar/Bronze/Recycling_Diversion_and_Capture_Rates.csv"

customSchema = StructType([
    StructField("zone", StringType(), True),
    StructField("district", StringType(), True),  
    StructField("fiscal_Month_Number", IntegerType(), True),
    StructField("fiscal_Year", IntegerType(), True),
    StructField("month_Name", StringType(), True),
    StructField("diversion_Rate_Total", DoubleType(), True),
    StructField("capture_Rate_Paper", DoubleType(), True),
    StructField("capture_Rate_Mgp", DoubleType(), True),
    StructField("capture_Rate_Total", DoubleType(), True)
])

sales_df = spark \
            .read \
            .option("delimiter",",") \
            .option("header","true") \
            .schema(customSchema) \
            .csv(hdfs_path)

sales_df.show(30)
#sales_df.printSchema()

+--------------+--------+-------------------+-----------+----------+--------------------+------------------+----------------+------------------+
|          zone|district|fiscal_Month_Number|fiscal_Year|month_Name|diversion_Rate_Total|capture_Rate_Paper|capture_Rate_Mgp|capture_Rate_Total|
+--------------+--------+-------------------+-----------+----------+--------------------+------------------+----------------+------------------+
|Brooklyn North|   BKN01|                 10|       2019|     April|                14.7|              44.9|            43.0|              44.1|
|Brooklyn North|   BKN02|                 10|       2019|     April|                20.0|              34.2|            57.9|              41.2|
|Brooklyn North|   BKN03|                 10|       2019|     April|                12.2|              33.5|            44.9|              38.2|
|Brooklyn North|   BKN04|                 10|       2019|     April|                15.5|              35.2|            68.5|     

In [14]:
from pyspark.sql.functions import when, col, concat, lit

sales_df1 = sales_df.withColumn(
    "zone",
    when((col("zone") == "Brooklyn North"), "Brooklyn").  
    when((col("zone") == "Brooklyn South"), "Brooklyn").
    when((col("zone") == "Queens West"), "Queens").
    when((col("zone") == "Queens East"), "Queens").
    otherwise(col("zone")))

sales_df1.toPandas()


zone district  fiscal_Month_Number  fiscal_Year month_Name  \
0          Brooklyn    BKN01                   10         2019      April   
1          Brooklyn    BKN02                   10         2019      April   
2          Brooklyn    BKN03                   10         2019      April   
3          Brooklyn    BKN04                   10         2019      April   
4          Brooklyn    BKN05                   10         2019      April   
...             ...      ...                  ...          ...        ...   
2827         Queens     QW06                    3         2016  September   
2828         Queens     QW09                    3         2016  September   
2829  Staten Island     SI01                    3         2016  September   
2830  Staten Island     SI02                    3         2016  September   
2831  Staten Island     SI03                    3         2016  September   

      diversion_Rate_Total  capture_Rate_Paper  capture_Rate_Mgp  \
0                     14.7                44.9              43.0   
1                     20.0                34.2              57.9   
2                     12.2                33.5              44.9   
3                     15.5                35.2              68.5   
4                     10.1                22.3              45.1   
...                    ...                 ...               ...   
2827                  20.1                30.4              68.0   
2828                  17.4                41.1              79.7   
2829                  18.7                39.5              71.7   
2830                  19.0                44.5              75.0   
2831                  20.3                48.2              79.5   

      capture_Rate_Total  
0                   44.1  
1                   41.2  
2                   38.2  
3                   48.8  
4                   31.5  
...                  ...  
2827                39.0  
2828                54.3  
2829                49.7  
2830                54.1  
2831                58.1  

[2832 rows x 9 columns]

In [15]:
sales_df1.select("zone").toPandas()

zone
0          Brooklyn
1          Brooklyn
2          Brooklyn
3          Brooklyn
4          Brooklyn
...             ...
2827         Queens
2828         Queens
2829  Staten Island
2830  Staten Island
2831  Staten Island

[2832 rows x 1 columns]

In [16]:
sales_df1.select('zone').count()

2832

In [17]:
from pyspark.sql.functions import when, col, concat, lit

sales_df2 = sales_df1.withColumn(
    "district",
    when((col("district") == "BKN01"), "Brooklyn Community Board 1").  
    when((col("district") == "BKN02"), "Brooklyn Community Board 2").
    when((col("district") == "BKN03"), "Brooklyn Community Board 3").
    when((col("district") == "BKN04"), "Brooklyn Community Board 4").
    when((col("district") == "BKN05"), "Brooklyn Community Board 5").  
    when((col("district") == "BKS06"), "Brooklyn Community Board 6").
    when((col("district") == "BKS07"), "Brooklyn Community Board 7").
    when((col("district") == "BKN08"), "Brooklyn Community Board 8").
    when((col("district") == "BKN09"), "Brooklyn Community Board 9").  
    when((col("district") == "BKS10"), "Brooklyn Community Board 10").
    when((col("district") == "BKS11"), "Brooklyn Community Board 11").
    when((col("district") == "BKS12"), "Brooklyn Community Board 12").
    when((col("district") == "BKS13"), "Brooklyn Community Board 13").  
    when((col("district") == "BKS14"), "Brooklyn Community Board 14").
    when((col("district") == "BKS15"), "Brooklyn Community Board 15").
    when((col("district") == "BKN16"), "Brooklyn Community Board 16").
    when((col("district") == "BKN17"), "Brooklyn Community Board 17").  
    when((col("district") == "BKS18"), "Brooklyn Community Board 18").
    when((col("district") == "BX01"), "Bronx Community Board 1").  
    when((col("district") == "BX02"), "Bronx Community Board 2").
    when((col("district") == "BX03"), "Bronx Community Board 3").
    when((col("district") == "BX04"), "Bronx Community Board 4").
    when((col("district") == "BX05"), "Bronx Community Board 5").  
    when((col("district") == "BX06"), "Bronx Community Board 6").
    when((col("district") == "BX07"), "Bronx Community Board 7").
    when((col("district") == "BX08"), "Bronx Community Board 8").
    when((col("district") == "BX09"), "Bronx Community Board 9").  
    when((col("district") == "BX10"), "Bronx Community Board 10").
    when((col("district") == "BX11"), "Bronx Community Board 11").
    when((col("district") == "BX12"), "Bronx Community Board 12").
    when((col("district") == "MN01"), "Manhattan Community Board 1").  
    when((col("district") == "MN02"), "Manhattan Community Board 2").
    when((col("district") == "MN03"), "Manhattan Community Board 3").
    when((col("district") == "MN04"), "Manhattan Community Board 4").
    when((col("district") == "MN05"), "Manhattan Community Board 5").  
    when((col("district") == "MN06"), "Manhattan Community Board 6").
    when((col("district") == "MN07"), "Manhattan Community Board 7").
    when((col("district") == "MN08"), "Manhattan Community Board 8").
    when((col("district") == "MN09"), "Manhattan Community Board 9").  
    when((col("district") == "MN10"), "Manhattan Community Board 10").
    when((col("district") == "MN11"), "Manhattan Community Board 11").
    when((col("district") == "MN12"), "Manhattan Community Board 12").
    when((col("district") == "QW01"), "Queens Community Board 1").  
    when((col("district") == "QW02"), "Queens Community Board 2").
    when((col("district") == "QW03"), "Queens Community Board 3").
    when((col("district") == "QW04"), "Queens Community Board 4").
    when((col("district") == "QW05"), "Queens Community Board 5").  
    when((col("district") == "QW06"), "Queens Community Board 6").
    when((col("district") == "QE07"), "Queens Community Board 7").
    when((col("district") == "QE08"), "Queens Community Board 8").
    when((col("district") == "QW09"), "Queens Community Board 9").  
    when((col("district") == "QE10"), "Queens Community Board 10").
    when((col("district") == "QE11"), "Queens Community Board 11").
    when((col("district") == "QE12"), "Queens Community Board 12").
    when((col("district") == "QE13"), "Queens Community Board 11").
    when((col("district") == "QE14"), "Queens Community Board 12").
    when((col("district") == "SI01"), "Stanten Island Community Board 1").
    when((col("district") == "SI02"), "Stanten Island Community Board 2").
    when((col("district") == "SI03"), "Stanten Island Community Board 3").
    otherwise(col("district")))

sales_df2.toPandas()

zone                          district  fiscal_Month_Number  \
0          Brooklyn        Brooklyn Community Board 1                   10   
1          Brooklyn        Brooklyn Community Board 2                   10   
2          Brooklyn        Brooklyn Community Board 3                   10   
3          Brooklyn        Brooklyn Community Board 4                   10   
4          Brooklyn        Brooklyn Community Board 5                   10   
...             ...                               ...                  ...   
2827         Queens          Queens Community Board 6                    3   
2828         Queens          Queens Community Board 9                    3   
2829  Staten Island  Stanten Island Community Board 1                    3   
2830  Staten Island  Stanten Island Community Board 2                    3   
2831  Staten Island  Stanten Island Community Board 3                    3   

      fiscal_Year month_Name  diversion_Rate_Total  capture_Rate_Paper  \
0            2019      April                  14.7                44.9   
1            2019      April                  20.0                34.2   
2            2019      April                  12.2                33.5   
3            2019      April                  15.5                35.2   
4            2019      April                  10.1                22.3   
...           ...        ...                   ...                 ...   
2827         2016  September                  20.1                30.4   
2828         2016  September                  17.4                41.1   
2829         2016  September                  18.7                39.5   
2830         2016  September                  19.0                44.5   
2831         2016  September                  20.3                48.2   

      capture_Rate_Mgp  capture_Rate_Total  
0                 43.0                44.1  
1                 57.9                41.2  
2                 44.9                38.2  
3                 68.5                48.8  
4                 45.1                31.5  
...                ...                 ...  
2827              68.0                39.0  
2828              79.7                54.3  
2829              71.7                49.7  
2830              75.0                54.1  
2831              79.5                58.1  

[2832 rows x 9 columns]

In [18]:
sales_df2.select("district").toPandas()

district
0           Brooklyn Community Board 1
1           Brooklyn Community Board 2
2           Brooklyn Community Board 3
3           Brooklyn Community Board 4
4           Brooklyn Community Board 5
...                                ...
2827          Queens Community Board 6
2828          Queens Community Board 9
2829  Stanten Island Community Board 1
2830  Stanten Island Community Board 2
2831  Stanten Island Community Board 3

[2832 rows x 1 columns]

In [19]:
sales_df2.select('district').count()

2832

In [20]:
sales_df3 = sales_df2.selectExpr("zone as Borough", "district as District", "fiscal_Month_Number as Fiscal_Month_Number", "fiscal_Year as Year", "month_Name as Month_Name", "diversion_Rate_Total as Diversion_Rate_Total",
                                        "capture_Rate_Paper as Capture_Rate_Paper", "capture_Rate_Mgp as Capture_Rate_MGP", "capture_Rate_Total as Capture_Rate_Total")

In [21]:
sales_df3.show()

+--------+--------------------+-------------------+----+----------+--------------------+------------------+----------------+------------------+
| Borough|            District|Fiscal_Month_Number|Year|Month_Name|Diversion_Rate_Total|Capture_Rate_Paper|Capture_Rate_MGP|Capture_Rate_Total|
+--------+--------------------+-------------------+----+----------+--------------------+------------------+----------------+------------------+
|Brooklyn|Brooklyn Communit...|                 10|2019|     April|                14.7|              44.9|            43.0|              44.1|
|Brooklyn|Brooklyn Communit...|                 10|2019|     April|                20.0|              34.2|            57.9|              41.2|
|Brooklyn|Brooklyn Communit...|                 10|2019|     April|                12.2|              33.5|            44.9|              38.2|
|Brooklyn|Brooklyn Communit...|                 10|2019|     April|                15.5|              35.2|            68.5|            

In [22]:
sales_df3 \
    .write \
    .mode("overwrite") \
    .partitionBy("Year") \
    .option("overwriteSchema", "true")\
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Silver/PL7G1_SILVER.db/Reciclagem") 
#.select ("zone", "district", "fiscal_Month_Number", "fiscal_Year", "month_Name", "diversion_Rate_Total", "capture_Rate_Paper", "capture_Rate_Mgp", "capture_Rate_Total") \
#"zone", "district", "fiscal_Month_Number", "fiscal_Year", "month_Name", "diversion_Rate_Total", "capture_Rate_Paper", "capture_Rate_Mgp", "capture_Rate_Total"
#"Zone", "District", "Fiscal_Month_Number", "Fiscal_Year", "Month_Name", "Diversion_Rate_Total", "Capture_Rate_Paper", "Capture_Rate_MGP", "Capture_Rate_Total"

In [23]:
spark.sql(
    """
    SELECT * FROM PL7G1_SILVER.Reciclagem
    """
).show()

+--------+--------------------+-------------------+----+----------+--------------------+------------------+----------------+------------------+
| Borough|            District|Fiscal_Month_Number|Year|Month_Name|Diversion_Rate_Total|Capture_Rate_Paper|Capture_Rate_MGP|Capture_Rate_Total|
+--------+--------------------+-------------------+----+----------+--------------------+------------------+----------------+------------------+
|Brooklyn|Brooklyn Communit...|                 10|2017|     April|                13.8|              42.3|            42.3|              42.3|
|Brooklyn|Brooklyn Communit...|                 10|2017|     April|                17.4|              33.5|            54.2|              39.6|
|Brooklyn|Brooklyn Communit...|                 10|2017|     April|                11.3|              31.7|            41.1|              35.5|
|Brooklyn|Brooklyn Communit...|                 10|2017|     April|                14.3|              31.5|            64.7|            